In [1]:
# import libraries for analysis
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import os

/var/folders/88/cfrnr91d1c74xcnth9yggzl80000gn/T/ipykernel_8498/820567876.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# define constants
DATA_DIR = 'data/experiment2' # directory where the data is stored
REPS = 30 # number of repetitions
LL_DELAYS = [5, 10, 20, 30, 40, 50, 60, 70, 80] # RIs used in the experiment
LL_MAG = 15 # FDF mean of the LL alternative

In [3]:
# define equations used in the analysis
def sigmoid(x, a, b, c, d):
    '''
    this is used to estimate the indifference points
    '''
    return a / (1 + np.exp(-(b * (x - c))))+d

def cubic(x, a, b, c, d):
    '''
    this is used for the cubic trend test (McDowell et al., 2016) to evaluate residuals 
    '''
    return a*x**3 + b*x**2 + c*x + d

# discounting models
# note: A must be defined before using these functions
def mazur(x, k):
    return A / (1 + k*x)

def rachlin(x, k, s):
    return A / (1 + k*x**s)

def myerson_green(x, k, s):
    return A / (1 + k*x)**s

In [11]:
# define functions for the analysis
def estimate_indifference_point(mags: np.ndarray, props: np.ndarray) -> float:
    '''
    this function fits the sigmoid function to the magnitudes and proportions of choices for the LL alternative and estimates the indifference point by finding the x value where the sigmoid function is 0.5
    '''
    params, _ = curve_fit(sigmoid, mags, props, p0=[-0.9, 0.05, 100, 1], maxfev=1000000)

    estimation_x = np.linspace(min(mags), max(mags), 10000)
    estimation_y = sigmoid(estimation_x, *params)

    indiff_pt = estimation_x[np.argmin(np.abs(estimation_y - 0.5))]

    return indiff_pt

def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    '''
    this function removes the first 500 generations of each schedule from the data
    '''
    
    cleaned_data = df.groupby(['File', 'Rep', 'Sched'])
    cleaned_data = cleaned_data.apply(lambda row: row.iloc[1:]).reset_index(drop=True)
    cleaned_data.sort_values(by=['SSMag', 'Rep', 'Sched'], inplace=True)

    return cleaned_data

def min_max_transform(indiff_pts: np.ndarray) -> np.ndarray:
    '''
    this function reverses the scale of the indifference points by subtracting each point from the sum of the minimum and maximum indifference points
    '''
    return ((min(indiff_pts) + max(indiff_pts)) - indiff_pts)

In [5]:
# load in the raw data from the excel files in the data directory

files = [f for f in os.listdir(DATA_DIR) if f.endswith('.xlsx')]
dfs = []
for f in files:
    for rep in range(REPS):
        data = pd.read_excel(os.path.join(DATA_DIR, f), sheet_name=f"Repetition {rep+1}")
        # add columns for the repetition, file, SS magnitude, LL delay
        data['Rep'] = rep + 1
        data['File'] = f
        data['SSMag'] = int(f.split('.')[0].replace('ssmag', ''))
        data['LLDelay'] = data['Sched'].apply(lambda x: int(LL_DELAYS[x-1]))

        dfs.append(data)

# concatenate the dataframes to create a single dataframe with all the raw data
raw_data = pd.concat(dfs)

In [6]:
raw_data

,Sched,P1,R1,B1,P2,R2,B2,Rep,File,SSMag,LLDelay
0,1,0,0,0,0,78,305,1,ssmag95.xlsx,95,5
1,1,0,0,0,0,68,340,1,ssmag95.xlsx,95,5
2,1,0,0,0,0,101,359,1,ssmag95.xlsx,95,5
3,1,0,1,1,0,71,291,1,ssmag95.xlsx,95,5
4,1,0,0,0,0,76,356,1,ssmag95.xlsx,95,5
...,...,...,...,...,...,...,...,...,...,...,...
364,9,0,111,137,0,0,0,30,ssmag75.xlsx,75,80
365,9,0,152,188,0,0,0,30,ssmag75.xlsx,75,80
366,9,0,148,191,0,0,0,30,ssmag75.xlsx,75,80
367,9,0,143,182,0,0,0,30,ssmag75.xlsx,75,80


In [12]:


cleaned_data = clean_data(raw_data)

cleaned_data

/var/folders/88/cfrnr91d1c74xcnth9yggzl80000gn/T/ipykernel_8498/3823808448.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cleaned_data = cleaned_data.apply(lambda row: row.iloc[1:]).reset_index(drop=True)


,Sched,P1,R1,B1,P2,R2,B2,Rep,File,SSMag,LLDelay
21600,1,0,267,429,0,0,0,1,ssmag15.xlsx,15,5
21601,1,0,282,441,0,0,0,1,ssmag15.xlsx,15,5
21602,1,0,275,402,0,0,0,1,ssmag15.xlsx,15,5
21603,1,0,258,405,0,0,0,1,ssmag15.xlsx,15,5
21604,1,0,280,406,0,0,0,1,ssmag15.xlsx,15,5
...,...,...,...,...,...,...,...,...,...,...,...
53995,9,0,10,11,0,3,44,30,ssmag170.xlsx,170,80
53996,9,0,61,65,0,1,12,30,ssmag170.xlsx,170,80
53997,9,0,43,48,0,4,56,30,ssmag170.xlsx,170,80
53998,9,0,31,33,0,3,54,30,ssmag170.xlsx,170,80


In [13]:
# get the mean response rate and reinforcer rate for each schedule for each repetition
ind_ao_data = cleaned_data.groupby(['File', 'Rep', 'Sched']).mean().reset_index()

ind_ao_data['PropSS'] = ind_ao_data['B1'] / (ind_ao_data['B1'] + ind_ao_data['B2']) # calculate the proportion of choices for the SS alternative

In [14]:
# get the mean and standard deviation of the response rate and reinforcer rate for each schedule across all repetitions
mean_sd_data = cleaned_data.groupby(['File', 'Sched']).agg(['mean', 'std']).reset_index()

mean_sd_data['PropSS'] = mean_sd_data[('B1', 'mean')] / (mean_sd_data[('B1', 'mean')] + mean_sd_data[('B2', 'mean')]) # calculate the mean proportion of choices for the SS alternative

In [31]:
# estimate the indifference points

# individual AO indifference points
ind_indiff_pts = {
    'Rep': [],
    'LLDelay': [],
    'IndiffPt': [],
    'TransformedIndiffPt': []
}
# estimate the indifference points for each repetition (AO) and delay
for rep in ind_ao_data['Rep'].unique():
    indiff_pts = []
    for delay in ind_ao_data['LLDelay'].unique():
        ind_data = ind_ao_data[(ind_ao_data['Rep'] == rep) & (ind_ao_data['LLDelay'] == delay)]
        indiff_pt = estimate_indifference_point(ind_data['SSMag'].values, ind_data['PropSS'].values)
        ind_indiff_pts['Rep'].append(rep)
        ind_indiff_pts['LLDelay'].append(delay)
        indiff_pts.append(indiff_pt)

    ind_indiff_pts['IndiffPt'].extend(indiff_pts)
    ind_indiff_pts['TransformedIndiffPt'].extend(min_max_transform(np.array(indiff_pts)))

# convert the dictionary to a dataframe
ind_indiff_pts = pd.DataFrame(ind_indiff_pts)

In [32]:
# estimate the indifference points for the mean data
mean_indiff_pts = {
    'LLDelay': [],
    'IndiffPt': [],
    'TransformedIndiffPt': []
}
# estimate the indifference points for each delay
indiff_pts = []
for ll_delay in mean_sd_data[('LLDelay', 'mean')].unique():
    mean_data = mean_sd_data[mean_sd_data[('LLDelay', 'mean')] == ll_delay]
    indiff_pt = estimate_indifference_point(mean_data[('SSMag', 'mean')].values, mean_data['PropSS'].values)
    mean_indiff_pts['LLDelay'].append(ll_delay)
    indiff_pts.append(indiff_pt)
    
# add the indifference points to the dictionary
mean_indiff_pts['IndiffPt'] = indiff_pts
mean_indiff_pts['TransformedIndiffPt'] = list(min_max_transform(np.array(indiff_pts)))

# convert the dictionary to a dataframe
mean_indiff_pts = pd.DataFrame(mean_indiff_pts)

In [33]:
# fit the discounting models to the individual AO indifference points

# create a dictionary to store the parameters of the discounting models
ind_fits_dict = {
    "Rep": [],
    "Mazur k": [],
    "Mazur R^2": [],
    "Mazur Sy.x": [],
    "Myerson & Green k": [],
    "Myerson & Green s": [],
    "Myerson & Green R^2": [],
    "Myerson & Green Sy.x": [],
    "Rachlin k": [],
    "Rachlin s": [],
    "Rachlin R^2": [],
    "Rachlin Sy.x": [],
}

for rep in ind_indiff_pts['Rep'].unique():
    rep_data = ind_indiff_pts[ind_indiff_pts['Rep'] == rep]

    # get the x and y values for the fitting
    x = rep_data['LLDelay'].values
    y = rep_data['TransformedIndiffPt'].values
    
    # calculate A for this AO
    A = min(y) + max(y) - LL_MAG

    # fit the Mazur model
    mazur_params, mazur_cov = curve_fit(mazur, x, y, p0=[0.05], maxfev=1000)

    # calculate the R^2 and Sy.x for the Mazur model
    residuals = y - mazur(x, *mazur_params)
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    r_squared = 1 - (ss_res / ss_tot)
    sy_x = np.sqrt(ss_res / (len(y) - 1))

    # fit the Myerson & Green model
    myerson_green_params, myerson_green_cov = curve_fit(myerson_green, x, y, p0=[0.05, 1], maxfev=1000)

    # calculate the R^2 and Sy.x for the Myerson & Green model
    residuals = y - myerson_green(x, *myerson_green_params)
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    r_squared = 1 - (ss_res / ss_tot)
    sy_x = np.sqrt(ss_res / (len(y) - 1))

    # fit the Rachlin model
    rachlin_params, rachlin_cov = curve_fit(rachlin, x, y, p0=[0.05, 1], maxfev=1000)

    # calculate the R^2 and Sy.x for the Rachlin model
    residuals = y - rachlin(x, *rachlin_params)
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    r_squared = 1 - (ss_res / ss_tot)
    sy_x = np.sqrt(ss_res / (len(y) - 1))

    # add the parameters to the dictionary
    ind_fits_dict['Rep'].append(rep)
    ind_fits_dict['Mazur k'].append(mazur_params[0])
    ind_fits_dict['Mazur R^2'].append(r_squared)
    ind_fits_dict['Mazur Sy.x'].append(sy_x)
    ind_fits_dict['Myerson & Green k'].append(myerson_green_params[0])
    ind_fits_dict['Myerson & Green s'].append(myerson_green_params[1])
    ind_fits_dict['Myerson & Green R^2'].append(r_squared)
    ind_fits_dict['Myerson & Green Sy.x'].append(sy_x)
    ind_fits_dict['Rachlin k'].append(rachlin_params[0])
    ind_fits_dict['Rachlin s'].append(rachlin_params[1])
    ind_fits_dict['Rachlin R^2'].append(r_squared)
    ind_fits_dict['Rachlin Sy.x'].append(sy_x)

# convert the dictionary to a dataframe
ind_fits = pd.DataFrame(ind_fits_dict)

/var/folders/88/cfrnr91d1c74xcnth9yggzl80000gn/T/ipykernel_8498/409966002.py:23: RuntimeWarning: invalid value encountered in power
  return A / (1 + k*x)**s


In [34]:
# fit the discounting models to the indifference points estimated from the mean data

mean_fits_dict = {
    "Mazur k": [],
    "Mazur R^2": [],
    "Mazur Sy.x": [],
    "Myerson & Green k": [],
    "Myerson & Green s": [],
    "Myerson & Green R^2": [],
    "Myerson & Green Sy.x": [],
    "Rachlin k": [],
    "Rachlin s": [],
    "Rachlin R^2": [],
    "Rachlin Sy.x": [],
}

# get the x and y values for the fitting
x = mean_indiff_pts['LLDelay'].values
y = mean_indiff_pts['TransformedIndiffPt'].values

# calculate A for the mean data
A = min(y) + max(y) - LL_MAG

# fit the Mazur model
mazur_params, mazur_cov = curve_fit(mazur, x, y, p0=[0.05], maxfev=1000)

# calculate the R^2 and Sy.x for the Mazur model
residuals = y - mazur(x, *mazur_params)
ss_res = np.sum(residuals**2)
ss_tot = np.sum((y - np.mean(y))**2)
r_squared = 1 - (ss_res / ss_tot)
sy_x = np.sqrt(ss_res / (len(y) - 1))

# fit the Myerson & Green model
myerson_green_params, myerson_green_cov = curve_fit(myerson_green, x, y, p0=[0.05, 1], maxfev=1000)

# calculate the R^2 and Sy.x for the Myerson & Green model
residuals = y - myerson_green(x, *myerson_green_params)
ss_res = np.sum(residuals**2)
ss_tot = np.sum((y - np.mean(y))**2)
r_squared = 1 - (ss_res / ss_tot)
sy_x = np.sqrt(ss_res / (len(y) - 1))
               
# fit the Rachlin model
rachlin_params, rachlin_cov = curve_fit(rachlin, x, y, p0=[0.05, 1], maxfev=1000)

# calculate the R^2 and Sy.x for the Rachlin model
residuals = y - rachlin(x, *rachlin_params)
ss_res = np.sum(residuals**2)
ss_tot = np.sum((y - np.mean(y))**2)
r_squared = 1 - (ss_res / ss_tot)
sy_x = np.sqrt(ss_res / (len(y) - 1))

# add the parameters to the dictionary
mean_fits_dict['Mazur k'].append(mazur_params[0])
mean_fits_dict['Mazur R^2'].append(r_squared)
mean_fits_dict['Mazur Sy.x'].append(sy_x)
mean_fits_dict['Myerson & Green k'].append(myerson_green_params[0])
mean_fits_dict['Myerson & Green s'].append(myerson_green_params[1])
mean_fits_dict['Myerson & Green R^2'].append(r_squared)
mean_fits_dict['Myerson & Green Sy.x'].append(sy_x)
mean_fits_dict['Rachlin k'].append(rachlin_params[0])
mean_fits_dict['Rachlin s'].append(rachlin_params[1])
mean_fits_dict['Rachlin R^2'].append(r_squared)
mean_fits_dict['Rachlin Sy.x'].append(sy_x)

# convert the dictionary to a dataframe
mean_fits = pd.DataFrame(mean_fits_dict)

In [35]:
# output the results to csv files
ind_ao_data.to_csv('ind_ao_data.csv', index=False)
mean_sd_data.to_csv('mean_sd_data.csv', index=False)
ind_indiff_pts.to_csv('ind_indiff_pts.csv', index=False)
mean_indiff_pts.to_csv('mean_indiff_pts.csv', index=False)
ind_fits.to_csv('ind_fits.csv', index=False)
mean_fits.to_csv('mean_fits.csv', index=False)